# Библиотеки

In [1]:
import math
from copy import deepcopy

import warnings
warnings.filterwarnings('ignore')

from catboost import CatBoostRegressor, cv, Pool

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
#!pip install plotly
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import graph_objs as go

# Подготовка данных для обучения

In [2]:
#размер валидационной выборки
N = 365*24 #4344

In [3]:
#datetime в правильный формат
def get_dt(x):
    date, time = x.split()
    day, month, year = date.split('.')
    return f"{year}.{month}.{day} {time}"

df = pd.read_csv('train.csv')
df['dt'] = pd.to_datetime(df.datetime.apply(get_dt))
df = df.set_index('dt')
df

,datetime,total
dt,,
2005-01-01 00:00:00,01.01.2005 00:00:00,?
2005-01-01 01:00:00,01.01.2005 01:00:00,154140
2005-01-01 02:00:00,01.01.2005 02:00:00,157818
2005-01-01 03:00:00,01.01.2005 03:00:00,149311
2005-01-01 04:00:00,01.01.2005 04:00:00,138282
...,...,...
2008-12-31 19:00:00,31.12.2008 19:00:00,249376
2008-12-31 20:00:00,31.12.2008 20:00:00,246511
2008-12-31 21:00:00,31.12.2008 21:00:00,226469


In [4]:
#удаление пропусков в данных
df = df[df['total'] != '?']
df['total'] = df['total'].astype(np.int64)
df = df[df['total'] > 0]

In [5]:
#построение графика
init_notebook_mode(connected = True)

def plotly_df(df, title = ''):
    data = []

    for column in df.columns:
        trace = go.Scatter(
            x = df.index,
            y = df[column],
            mode = 'lines',
            name = column
        )
        data.append(trace)

    layout = dict(title = title)
    fig = dict(data = data, layout = layout)
    iplot(fig, show_link=False)

plotly_df(df.resample('D').apply(sum)[['total']], title = 'Потреблении воды в день. All.')

In [6]:
#число дней до текущего года
pref_days = [0]
for year in range(2005, 2010):
    pref_days.append(pd.Timestamp(f'{year}-12-31').dayofyear)
print(pref_days)
for i in range(1, len(pref_days)):
    pref_days[i] += pref_days[i-1]
print(pref_days)

[0, 365, 365, 365, 366, 365]
[0, 365, 730, 1095, 1461, 1826]


In [7]:
#признаки из datetime
def prepare(x):
    x = x.split(' ')
    date = x[0].split('.')
    time = x[1].split(':')
    day, month, year = map(int, date)
    timestamp = pd.Timestamp(f'{year}-{month}-{day}')
    hour = int(time[0])
    dayofyear = timestamp.dayofyear
    dayofweek = timestamp.dayofweek
    ind_date = dayofyear + pref_days[year - 2005] - 1
    ind_hour = ind_date * 24 + hour
    if dayofweek >= 5:
        days_to_weekend = 0
    else:
        days_to_weekend = min(dayofweek + 1, abs(5 - dayofweek))
    return year, month, day, hour, dayofyear, dayofweek, dayofweek>=5, (month-1) // 3, ind_date, ind_hour, days_to_weekend

add_columns = ['year', 'month', 'day', 'hour', 'dayofyear', 'dayofweek', 'is_weekend', \
               'season', 'ind_date', 'ind_hour', 'days_to_weekend']
df[add_columns] = list(df['datetime'].apply(prepare))
for col in add_columns:
    df[col] = df[col].astype(np.int32)
df = df.sort_values(by='ind_hour')
df

,datetime,total,year,month,day,hour,dayofyear,dayofweek,is_weekend,season,ind_date,ind_hour,days_to_weekend
dt,,,,,,,,,,,,,
2005-01-01 01:00:00,01.01.2005 01:00:00,154140,2005,1,1,1,1,5,1,0,0,1,0
2005-01-01 02:00:00,01.01.2005 02:00:00,157818,2005,1,1,2,1,5,1,0,0,2,0
2005-01-01 03:00:00,01.01.2005 03:00:00,149311,2005,1,1,3,1,5,1,0,0,3,0
2005-01-01 04:00:00,01.01.2005 04:00:00,138282,2005,1,1,4,1,5,1,0,0,4,0
2005-01-01 05:00:00,01.01.2005 05:00:00,132033,2005,1,1,5,1,5,1,0,0,5,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-31 19:00:00,31.12.2008 19:00:00,249376,2008,12,31,19,366,2,0,3,1460,35059,3
2008-12-31 20:00:00,31.12.2008 20:00:00,246511,2008,12,31,20,366,2,0,3,1460,35060,3
2008-12-31 21:00:00,31.12.2008 21:00:00,226469,2008,12,31,21,366,2,0,3,1460,35061,3


In [8]:
#добавление тригонометрических функций от данных
def add_trigonom(df):
    df['sin_month'] = df['month'].apply(math.sin)
    df['cos_month'] = df['month'].apply(math.cos)
    df['sin_hour'] = df['hour'].apply(math.sin)
    df['cos_hour'] = df['hour'].apply(math.cos)
    df['sin**2_hour'] = df['sin_hour'] * df['sin_hour']
    df['cos**2_hour'] = df['cos_hour'] * df['cos_hour']
    
    df['hour_sin'] = (df['hour'] / 23 * 2 * np.pi).apply(math.sin)
    df['hour_cos'] = (df['hour'] / 23 * 2 * np.pi).apply(math.cos)
    
    df['month_sin'] = ((df['month'] - 1) / 11 * 2 * np.pi).apply(math.sin)
    df['month_cos'] = ((df['month'] - 1) / 11 * 2 * np.pi).apply(math.cos)
    
    df['hour_sin**2'] = df['hour_sin'] * df['hour_sin']
    df['month_cos**2'] = df['month_cos'] * df['month_cos']
    
    day = 24
    year = 365.2425*day
    df['day_sin'] = (df.index.hour * 2 * np.pi / day)
    df['day_sin'] = df['day_sin'].apply(math.sin)
    df['day_cos'] = (df.index.hour * 2 * np.pi / day)
    df['day_cos'] = df['day_cos'].apply(math.cos)
    df['year_sin'] = (df.index.hour * 2 * np.pi / year)
    df['year_sin'] = df['year_sin'].apply(math.sin)
    df['year_cos'] = (df.index.hour * 2 * np.pi / year)
    df['year_cos'] = df['year_cos'].apply(math.cos)
    
    return df

df = add_trigonom(df)
df

,datetime,total,year,month,day,hour,dayofyear,dayofweek,is_weekend,season,...,hour_sin,hour_cos,month_sin,month_cos,hour_sin**2,month_cos**2,day_sin,day_cos,year_sin,year_cos
dt,,,,,,,,,,,,,,,,,,,,,
2005-01-01 01:00:00,01.01.2005 01:00:00,154140,2005,1,1,1,1,5,1,0,...,2.697968e-01,0.962917,0.000000e+00,1.0,7.279030e-02,1.0,0.258819,0.965926,0.000717,1.000000
2005-01-01 02:00:00,01.01.2005 02:00:00,157818,2005,1,1,2,1,5,1,0,...,5.195840e-01,0.854419,0.000000e+00,1.0,2.699675e-01,1.0,0.500000,0.866025,0.001434,0.999999
2005-01-01 03:00:00,01.01.2005 03:00:00,149311,2005,1,1,3,1,5,1,0,...,7.308360e-01,0.682553,0.000000e+00,1.0,5.341212e-01,1.0,0.707107,0.707107,0.002150,0.999998
2005-01-01 04:00:00,01.01.2005 04:00:00,138282,2005,1,1,4,1,5,1,0,...,8.878852e-01,0.460065,0.000000e+00,1.0,7.883402e-01,1.0,0.866025,0.500000,0.002867,0.999996
2005-01-01 05:00:00,01.01.2005 05:00:00,132033,2005,1,1,5,1,5,1,0,...,9.790841e-01,0.203456,0.000000e+00,1.0,9.586057e-01,1.0,0.965926,0.258819,0.003584,0.999994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-31 19:00:00,31.12.2008 19:00:00,249376,2008,12,31,19,366,2,0,3,...,-8.878852e-01,0.460065,-2.449294e-16,1.0,7.883402e-01,1.0,-0.965926,0.258819,0.013618,0.999907
2008-12-31 20:00:00,31.12.2008 20:00:00,246511,2008,12,31,20,366,2,0,3,...,-7.308360e-01,0.682553,-2.449294e-16,1.0,5.341212e-01,1.0,-0.866025,0.500000,0.014335,0.999897
2008-12-31 21:00:00,31.12.2008 21:00:00,226469,2008,12,31,21,366,2,0,3,...,-5.195840e-01,0.854419,-2.449294e-16,1.0,2.699675e-01,1.0,-0.707107,0.707107,0.015052,0.999887


In [9]:
#one-hot кодирование категориальных признаков
df = pd.get_dummies(df, columns=['is_weekend', 'season'])
df.dropna(inplace=True)
df

,datetime,total,year,month,day,hour,dayofyear,dayofweek,ind_date,ind_hour,...,day_sin,day_cos,year_sin,year_cos,is_weekend_0,is_weekend_1,season_0,season_1,season_2,season_3
dt,,,,,,,,,,,,,,,,,,,,,
2005-01-01 01:00:00,01.01.2005 01:00:00,154140,2005,1,1,1,1,5,0,1,...,0.258819,0.965926,0.000717,1.000000,0,1,1,0,0,0
2005-01-01 02:00:00,01.01.2005 02:00:00,157818,2005,1,1,2,1,5,0,2,...,0.500000,0.866025,0.001434,0.999999,0,1,1,0,0,0
2005-01-01 03:00:00,01.01.2005 03:00:00,149311,2005,1,1,3,1,5,0,3,...,0.707107,0.707107,0.002150,0.999998,0,1,1,0,0,0
2005-01-01 04:00:00,01.01.2005 04:00:00,138282,2005,1,1,4,1,5,0,4,...,0.866025,0.500000,0.002867,0.999996,0,1,1,0,0,0
2005-01-01 05:00:00,01.01.2005 05:00:00,132033,2005,1,1,5,1,5,0,5,...,0.965926,0.258819,0.003584,0.999994,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-31 19:00:00,31.12.2008 19:00:00,249376,2008,12,31,19,366,2,1460,35059,...,-0.965926,0.258819,0.013618,0.999907,1,0,0,0,0,1
2008-12-31 20:00:00,31.12.2008 20:00:00,246511,2008,12,31,20,366,2,1460,35060,...,-0.866025,0.500000,0.014335,0.999897,1,0,0,0,0,1
2008-12-31 21:00:00,31.12.2008 21:00:00,226469,2008,12,31,21,366,2,1460,35061,...,-0.707107,0.707107,0.015052,0.999887,1,0,0,0,0,1


In [10]:
#данные для обучения
df_train = df[:-N].copy()
df_train

,datetime,total,year,month,day,hour,dayofyear,dayofweek,ind_date,ind_hour,...,day_sin,day_cos,year_sin,year_cos,is_weekend_0,is_weekend_1,season_0,season_1,season_2,season_3
dt,,,,,,,,,,,,,,,,,,,,,
2005-01-01 01:00:00,01.01.2005 01:00:00,154140,2005,1,1,1,1,5,0,1,...,2.588190e-01,0.965926,0.000717,1.000000,0,1,1,0,0,0
2005-01-01 02:00:00,01.01.2005 02:00:00,157818,2005,1,1,2,1,5,0,2,...,5.000000e-01,0.866025,0.001434,0.999999,0,1,1,0,0,0
2005-01-01 03:00:00,01.01.2005 03:00:00,149311,2005,1,1,3,1,5,0,3,...,7.071068e-01,0.707107,0.002150,0.999998,0,1,1,0,0,0
2005-01-01 04:00:00,01.01.2005 04:00:00,138282,2005,1,1,4,1,5,0,4,...,8.660254e-01,0.500000,0.002867,0.999996,0,1,1,0,0,0
2005-01-01 05:00:00,01.01.2005 05:00:00,132033,2005,1,1,5,1,5,0,5,...,9.659258e-01,0.258819,0.003584,0.999994,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007-12-29 10:00:00,29.12.2007 10:00:00,215998,2007,12,29,10,363,5,1092,26218,...,5.000000e-01,-0.866025,0.007168,0.999974,0,1,0,0,0,1
2007-12-29 11:00:00,29.12.2007 11:00:00,213937,2007,12,29,11,363,5,1092,26219,...,2.588190e-01,-0.965926,0.007885,0.999969,0,1,0,0,0,1
2007-12-29 12:00:00,29.12.2007 12:00:00,211923,2007,12,29,12,363,5,1092,26220,...,1.224647e-16,-1.000000,0.008601,0.999963,0,1,0,0,0,1


In [11]:
#данные для валидации
df_val = df[-N:].copy()
df_val

,datetime,total,year,month,day,hour,dayofyear,dayofweek,ind_date,ind_hour,...,day_sin,day_cos,year_sin,year_cos,is_weekend_0,is_weekend_1,season_0,season_1,season_2,season_3
dt,,,,,,,,,,,,,,,,,,,,,
2007-12-29 15:00:00,29.12.2007 15:00:00,200548,2007,12,29,15,363,5,1092,26223,...,-0.707107,-7.071068e-01,0.010752,0.999942,0,1,0,0,0,1
2007-12-29 16:00:00,29.12.2007 16:00:00,198625,2007,12,29,16,363,5,1092,26224,...,-0.866025,-5.000000e-01,0.011468,0.999934,0,1,0,0,0,1
2007-12-29 17:00:00,29.12.2007 17:00:00,199867,2007,12,29,17,363,5,1092,26225,...,-0.965926,-2.588190e-01,0.012185,0.999926,0,1,0,0,0,1
2007-12-29 18:00:00,29.12.2007 18:00:00,201798,2007,12,29,18,363,5,1092,26226,...,-1.000000,-1.836970e-16,0.012902,0.999917,0,1,0,0,0,1
2007-12-29 19:00:00,29.12.2007 19:00:00,207616,2007,12,29,19,363,5,1092,26227,...,-0.965926,2.588190e-01,0.013618,0.999907,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2008-12-31 19:00:00,31.12.2008 19:00:00,249376,2008,12,31,19,366,2,1460,35059,...,-0.965926,2.588190e-01,0.013618,0.999907,1,0,0,0,0,1
2008-12-31 20:00:00,31.12.2008 20:00:00,246511,2008,12,31,20,366,2,1460,35060,...,-0.866025,5.000000e-01,0.014335,0.999897,1,0,0,0,0,1
2008-12-31 21:00:00,31.12.2008 21:00:00,226469,2008,12,31,21,366,2,1460,35061,...,-0.707107,7.071068e-01,0.015052,0.999887,1,0,0,0,0,1


In [12]:
#нахождение тренда
a, b = np.polyfit(list(df_train['ind_hour']), list(df_train['total']), 1)
#удаление тренда
df_train['total'] = df_train['total'] - a * df_train['ind_hour']
df_val['total'] = df_val['total'] - a * df_val['ind_hour']

In [13]:
trend_df = pd.DataFrame({'dt': df.index, 'init_total': df['total'], 'no_trend_total': pd.concat([df_train['total'], df_val['total']])}).set_index('dt')
#строим график
plotly_df(trend_df.resample('W').apply(sum), title = 'Потреблении воды по неделям. All.')

# Обучение модели

In [14]:
train_pool = Pool(
    data=df_train.drop(['datetime', 'total'], axis = 1),
    label=df_train['total'],
)
val_pool = Pool(
    data=df_val.drop(['datetime', 'total'], axis = 1),
    label=df_val['total'],
)

In [15]:
params = {
    'task_type': 'CPU', 
    'loss_function': 'MAE',
    'max_depth': 5,
    'iterations': 6000,
    'learning_rate': 0.03
}

In [16]:
model_cb = CatBoostRegressor(**params, random_seed=56)
model_cb.fit(train_pool, eval_set=val_pool, plot = True, verbose = False, use_best_model=True)
best_it = np.argmin(model_cb.evals_result_['validation']['MAE']) + 1
imp = model_cb.get_feature_importance(prettified=True).set_index('Feature Id')
imp

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

,Importances
Feature Id,
hour,25.343720
year_cos,23.741876
year_sin,10.246702
day_cos,5.135267
hour_sin,4.576174
month_cos,4.372671
cos_hour,4.173898
day_sin,3.475090
dayofweek,3.286928


In [17]:
#считаем метрику на валидации
def calc(y_true, y_pred):
    return 1000 * (1 - np.mean(abs(y_true - y_pred) / (y_true + y_pred)))
y_true_val = (val_pool.get_label() + a * df_val['ind_hour'])
y_pred_val = (model_cb.predict(val_pool) + a * df_val['ind_hour'])
print(calc(y_true_val, y_pred_val))

982.5134027132542


In [18]:
#строим график
ans_df = pd.DataFrame({'dt': df_val.index, 'true': y_true_val, 'pred': y_pred_val}).set_index('dt')
plotly_df(ans_df.resample('D').apply(sum), title = 'Потреблении воды по дням. Val.')

# Предсказание

In [19]:
#строим датасет для теста
df_test = []

year = 2009
for month in range(1, 7):
    for day in range(1, pd.Timestamp(f'{year}-0{month}-01').daysinmonth + 1):
        if month == 6 and day > 30:
            break
            
        for hour in range(0, 24):
            s_year = str(year)
            if month < 10:
                s_month = "0" + str(month)
            else:
                s_month = str(month)
            if day < 10:
                s_day = "0" + str(day)
            else:
                s_day = str(day)
            if hour < 10:
                s_hour = "0" + str(hour)
            else:
                s_hour = str(hour)
            datetime = f"{s_day}.{s_month}.{s_year} {s_hour}:00:00"
            dt = f"{s_year}.{s_month}.{s_day} {s_hour}:00:00"
            
            df_test.append({
                'dt': dt,
                'datetime': datetime
            })
df_test = pd.DataFrame(df_test)
#аналогично добавляем признаки
df_test['dt'] = pd.to_datetime(df_test.datetime.apply(get_dt))
df_test = df_test.set_index('dt')
df_test[add_columns] = list(df_test['datetime'].apply(prepare))
df_test=pd.get_dummies(df_test, columns=['is_weekend', 'season'])
df_test = add_trigonom(df_test)
for col in df.columns:
    if col not in df_test:
        df_test[col] = [0] * len(df_test)
df_test

,datetime,year,month,day,hour,dayofyear,dayofweek,ind_date,ind_hour,days_to_weekend,...,month_cos**2,day_sin,day_cos,year_sin,year_cos,total,is_weekend_0,is_weekend_1,season_2,season_3
dt,,,,,,,,,,,,,,,,,,,,,
2009-01-01 00:00:00,01.01.2009 00:00:00,2009,1,1,0,1,3,1461,35064,2,...,1.000000,0.000000,1.000000,0.000000,1.000000,0,0,0,0,0
2009-01-01 01:00:00,01.01.2009 01:00:00,2009,1,1,1,1,3,1461,35065,2,...,1.000000,0.258819,0.965926,0.000717,1.000000,0,0,0,0,0
2009-01-01 02:00:00,01.01.2009 02:00:00,2009,1,1,2,1,3,1461,35066,2,...,1.000000,0.500000,0.866025,0.001434,0.999999,0,0,0,0,0
2009-01-01 03:00:00,01.01.2009 03:00:00,2009,1,1,3,1,3,1461,35067,2,...,1.000000,0.707107,0.707107,0.002150,0.999998,0,0,0,0,0
2009-01-01 04:00:00,01.01.2009 04:00:00,2009,1,1,4,1,3,1461,35068,2,...,1.000000,0.866025,0.500000,0.002867,0.999996,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2009-06-30 19:00:00,30.06.2009 19:00:00,2009,6,30,19,181,1,1641,39403,2,...,0.920627,-0.965926,0.258819,0.013618,0.999907,0,0,0,0,0
2009-06-30 20:00:00,30.06.2009 20:00:00,2009,6,30,20,181,1,1641,39404,2,...,0.920627,-0.866025,0.500000,0.014335,0.999897,0,0,0,0,0
2009-06-30 21:00:00,30.06.2009 21:00:00,2009,6,30,21,181,1,1641,39405,2,...,0.920627,-0.707107,0.707107,0.015052,0.999887,0,0,0,0,0


In [20]:
#обучаем модель на полном датасете
a, b = np.polyfit(list(df['ind_hour']), list(df['total']), 1)
df['total'] = df['total'] - a * df['ind_hour']

pool = Pool(
    data=df.drop(['datetime', 'total'], axis = 1),
    label=df['total'],
)

new_params = deepcopy(params)
new_params['iterations'] = best_it

model_cb = CatBoostRegressor(**new_params, random_seed=56)
model_cb.fit(pool, plot = True, verbose = False)
model_cb.get_feature_importance(prettified=True).set_index('Feature Id')

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

,Importances
Feature Id,
hour,24.705834
year_cos,21.714069
year_sin,11.596008
month_cos,4.920375
hour_sin,4.832143
cos_hour,4.466162
dayofweek,4.425235
day_cos,4.031958
day_sin,4.027895


In [21]:
#получаем предскзаание для теста
preds = model_cb.predict(df_test.drop('datetime', axis = 1)) + a * df_test['ind_hour']

In [22]:
#строим график
ans_df = pd.DataFrame({'dt': df_test.index, 'pred': preds}).set_index('dt')
plotly_df(ans_df.resample('D').apply(sum), title = 'Потреблении воды по дням. Test.')

In [23]:
#сохраняем ответ
with open(f'answer.txt', 'w') as f:
    for v in preds:
        f.write(str(v))
        f.write('\n')